In [1]:
from pytorch_libraries.model_NeuralStyleTransfer import *
from IPython.display import clear_output
from pylab import *
%matplotlib inline

In [6]:
# Image preprocessing
# For normalization, see https://github.com/pytorch/vision#models
transform = transforms.Compose([transforms.Resize(256),
                                transforms.ToTensor()])
style   = Variable(load_image('../data/images/s2.jpg', transform))
content = Variable(load_image('../data/images/c2.jpg', transform))
target  = Variable(content.data.clone(), requires_grad=True)

vgg     = VGGNet().cuda()

In [15]:
optimizer = torch.optim.Adam([target], lr=5e-3)
    
STYLE_WEIGHT = 1e4
TOTAL_OPTSTEP= 100

In [ ]:
 for step in range(TOTAL_OPTSTEP):
        
        # Extract multiple(5) conv feature vectors
        target_features  = vgg(target)
        content_features = vgg(content)
        style_features   = vgg(style)

        style_loss = 0
        content_loss = 0
        
        # for each layer
        for f1, f2, f3 in zip(target_features, content_features, style_features):
            # Compute content loss (target and content image)
            content_loss += torch.mean((f1 - f2)**2)

            # Reshape conv features
            _, c, h, w = f1.size()
            f1 = f1.view(c, h * w)
            f3 = f3.view(c, h * w)
            # Compute gram matrix  
            f1 = torch.mm(f1, f1.t())
            f3 = torch.mm(f3, f3.t())
            # Compute style loss (target and style image)
            style_loss += torch.mean((f1 - f3)**2) / (c * h * w) 

        # Compute total loss, backprop and optimize
        loss = content_loss + STYLE_WEIGHT * style_loss 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (step+1) % 10 == 0:
            print ('Step [%d/%d], Content Loss: %.4f, Style Loss: %.4f' 
                   %(step+1, TOTAL_OPTSTEP, content_loss.data[0], style_loss.data[0]))
    
        if (step+1) % 10 == 0:
            # Save the generated image
            #denorm = transforms.Normalize((-2.12, -2.04, -1.80), (4.37, 4.46, 4.44))
            img = target.clone().cpu().squeeze()
            img = img.data.clamp_(0, 1)
            torchvision.utils.save_image(img, 'output-%d.png' %(step+1))

Step [10/100], Content Loss: 10.0895, Style Loss: 0.0659
Step [20/100], Content Loss: 10.2163, Style Loss: 0.0452
Step [30/100], Content Loss: 10.1798, Style Loss: 0.0380
Step [40/100], Content Loss: 10.1741, Style Loss: 0.0352
Step [50/100], Content Loss: 10.1874, Style Loss: 0.0337
Step [60/100], Content Loss: 10.2080, Style Loss: 0.0326
